In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.manifold import TSNE
from vae import VariationalAutoencoder
from modules import train_epoch, test_epoch, plot_ae_outputs


/home/szaryvip/INZ/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

Selected device: cuda


In [3]:
# dataset = torchvision.datasets.ImageFolder("../media/")
dataset = torchvision.datasets.MNIST(root="", train=True, download=True, transform=None)
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset.transform = train_transform

m=len(dataset)
trainset, valset = random_split(dataset, [int(m-0.2*m), int(0.2*m)])


train_loader = DataLoader(trainset)
val_loader = DataLoader(valset)

In [4]:
in_channels = len(dataset[0][0])
d = 128
lr = 1e-3 
model = VariationalAutoencoder(in_channels, d, device)
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
model.to(device)

VariationalAutoencoder(
  (encoder): VariationalEncoder(
    (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (batch2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (linear1): Linear(in_features=32768, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=128, bias=True)
    (linear3): Linear(in_features=128, out_features=128, bias=True)
  )
  (decoder): Decoder(
    (decoder_lin): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=128, out_features=32768, bias=True)
      (3): ReLU(inplace=True)
    )
    (unflatten): Unflatten(dim=1, unflattened_size=(32, 32, 32))
    (decoder_conv): Sequential(
      (0): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2))
     

In [5]:
num_epochs = 100

for epoch in range(num_epochs):
   train_loss = train_epoch(model,device,train_loader,optim,criterion)
   val_loss = test_epoch(model,device,val_loader,criterion)
   print('\n EPOCH {}/{} \t train loss {:.3f} \t val loss {:.3f}'.format(epoch + 1, num_epochs,train_loss,val_loss))
   plot_ae_outputs(model.encoder,model.decoder,valset,device,n=2)

	 partial train loss (single batch): 252.370392
	 partial train loss (single batch): 481.034576
	 partial train loss (single batch): 389.071411
	 partial train loss (single batch): 315.202637
	 partial train loss (single batch): 263.475830
	 partial train loss (single batch): 256.638611
	 partial train loss (single batch): 231.902130
	 partial train loss (single batch): 234.100662
	 partial train loss (single batch): 224.211044
	 partial train loss (single batch): 235.482803
	 partial train loss (single batch): 228.238113
	 partial train loss (single batch): 227.026962
	 partial train loss (single batch): 235.561829
	 partial train loss (single batch): 213.398529
	 partial train loss (single batch): 218.982071
	 partial train loss (single batch): 210.975006
	 partial train loss (single batch): 214.879150
	 partial train loss (single batch): 216.238037
	 partial train loss (single batch): 207.698929
	 partial train loss (single batch): 204.032684
	 partial train loss (single batch): 205

KeyboardInterrupt: 